
# Amazon Bedrock Guardrails with Crew AI Agents
Building safe and responsible AI applications is crucial, especially in sensitive domains like finance, healthcare, and customer service. Agentic AI systems can benefit significantly from guardrails that help ensure compliance with safety and ethical standards.

[Crew AI](https://docs.crewai.com/introduction) is python based multi-agent framework that empowers developers with both high-level simplicity and precise low-level control, ideal for creating autonomous AI agents tailored to any scenario.

[Amazon Bedrock Guardrails](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails.html) provide safety mechanisms that help control AI system behavior by defining boundaries for content generation and interaction. The Strands Agents SDK offers seamless integration with these guardrails, enabling you to implement:

* **Content filtering** - Block harmful or inappropriate content
* **Topic blocking** - Prevent discussions on specific topics
* **PII protection** - Detect and handle personally identifiable information
* **Word and phrase filtering** - Control specific language in interactions
* **Contextual grounding** - Ensure responses are relevant and factual

In this notebook, we will see how to integrate Amazon Bedrock Guardrails with Crew AI to ensure safe and responsible AI interactions in a banking assistant application.

## 1. Setup

### 1.1 Install the libraries

In [1]:
import os
import sys
import argparse
from typing import Dict, List, Any, Tuple
import json
import random

# AWS SDK
import boto3
from botocore.config import Config
import logging

# CrewAI imports
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool

# New tool imports from DataCamp example
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool

## 1.2 Set up the bedrock client 

In [4]:
# Set up Bedrock client with appropriate configuration
bedrock_config = Config(
    region_name="us-east-1",
    retries={"max_attempts": 3, "mode": "standard"}
)

client = boto3.client("bedrock")



## 2. Create a Guardrail
We will now create our first Guardrail, which will prevent the model from providing fiduciary advice. Guardrails for Amazon Bedrock have multiple components which include Content Filters, Denied Topics, Word and Phrase Filters, and Sensitive Word (PII & Regex) Filters. You can also review the [API documentation](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_CreateGuardrail.html#bedrock-CreateGuardrail-request-contextualGroundingPolicyConfig) for more details.

In [ ]:
#If the guardrail already exists, it will return the existing guardrail.
list_response = client.list_guardrails(maxResults=10)

exists = False
for guardrail in list_response.get('guardrails', []):
    if guardrail['name'] == 'financial-advice':
        print(f"Guardrail already exists: {guardrail['id']}")
        guardrail_id = guardrail['id']
        exists = True
if not exists:

    create_response = client.create_guardrail(
        name='financial-advice',
        description='Prevents the our model from providing financial advice.',
        topicPolicyConfig={
            'topicsConfig': [
                {
                    'name': 'Fiduciary Advice',
                    'definition': 'Provide advice on financial investments.',
                    'examples': [
                        'What stocks should I invest in for my retirement?',
                        'Is it a good idea to put my money in a mutual fund?',
                        'How should I allocate my 401(k) investments?',
                        'What type of trust fund should I set up for my children?',
                        'Should I hire a financial advisor to manage my investments?'
                    ],
                    'type': 'DENY'
                }
            ]
        },
        contentPolicyConfig={
            'filtersConfig': [
                {
                    'type': 'SEXUAL',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'HIGH'
                },
                {
                    'type': 'VIOLENCE',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'HIGH'
                },
                {
                    'type': 'HATE',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'HIGH'
                },
                {
                    'type': 'INSULTS',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'HIGH'
                },
                {
                    'type': 'MISCONDUCT',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'HIGH'
                },
                {
                    'type': 'PROMPT_ATTACK',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'NONE'
                }
            ]
        },
        wordPolicyConfig={
            'wordsConfig': [
                {'text': 'fiduciary advice'},
                {'text': 'investment recommendations'},
                {'text': 'stock picks'},
                {'text': 'financial planning guidance'},
                {'text': 'portfolio allocation advice'},
                {'text': 'retirement fund suggestions'},
                {'text': 'wealth management tips'},
                {'text': 'trust fund setup'},
                {'text': 'investment strategy'},
                {'text': 'financial advisor recommendations'}
            ],
            'managedWordListsConfig': [
                {'type': 'PROFANITY'}
            ]
        },
        sensitiveInformationPolicyConfig={
            'piiEntitiesConfig': [
                {'type': 'EMAIL', 'action': 'ANONYMIZE'},
                {'type': 'PHONE', 'action': 'ANONYMIZE'},
                {'type': 'NAME', 'action': 'ANONYMIZE'},
                {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
                {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
                {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
            ],
            'regexesConfig': [
                {
                    'name': 'Account Number',
                    'description': 'Matches account numbers in the format XXX123456',
                    'pattern': r'\b\d{6}\d{4}\b',
                    'action': 'ANONYMIZE'
                }
            ]
        },
        contextualGroundingPolicyConfig={
            'filtersConfig': [
                {
                    'type': 'GROUNDING',
                    'threshold': 0.75
                },
                {
                    'type': 'RELEVANCE',
                    'threshold': 0.75
                }
            ]
        },
        blockedInputMessaging="""Sorry, contact our customer service for this request.""",
        blockedOutputsMessaging="""Sorry, contact our customer service for this request.""",
        tags=[
            {'key': 'purpose', 'value': 'fiduciary-advice-prevention'},
            {'key': 'environment', 'value': 'production'}
        ]
    )

    print(create_response)
    guardrail_id = create_response['guardrailId']

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/botocore/crt/auth.py:59: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow().replace(


Guardrail already exists: dc1j2a3w9582


## 3. Create Apply Guardrails Function
The Apply Guardrails API allows us to use Amazon Bedrock Guardrail with any model or framework. We will now define a function to use this API for all inputs that will go into our agentic system.

In [9]:
def apply_guardrails(content: str) -> Tuple[str, Dict]:
    """Apply AWS Bedrock Guardrails to content"""
    try:
        print(f"Applying guardrail to content: {content[:50]}...")
        
        bedrock_client = boto3.client(
            "bedrock-runtime",
            config=bedrock_config,
            aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
            aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY")
        )
        
        response = bedrock_client.apply_guardrail(
            guardrailIdentifier=guardrail_id,
            guardrailVersion="DRAFT",
            source="INPUT",
            content=[
                {
                    "text": {
                        "text": content
                    }
                }
            ]
        )
        
        # Parse response
        outputs = response.get('outputs', [])
        if outputs and len(outputs) > 0:
            guardrailed_content = outputs[0].get('text', content)
        else:
            guardrailed_content = content
        
        # Get guardrail action
        action = response.get('action', 'NONE')
        
        # Create metadata dictionary
        metadata_dict = {
            "action": action,
            "assessments": response.get('assessments', [])
        }
        
        return guardrailed_content, metadata_dict
    
    except Exception as e:
        print(f"Error applying guardrails: {e}")
        return content, {"action": "ERROR", "error": str(e), "assessments": []}


## 4. Create the tools for our agent
We will now create the tools that our agent will be able to access during the conversation. In this case, we are creating a Banking assistant agent, therefore we will create the following tools:
* `get_balance`: A tool to get the balance of a bank account.
* `find_branch`: A tool to find the nearest bank branch.
* `check_loan_status`: A tool to check the status of a loan application.


In [10]:
from db_build import setup_bank_database
import sqlite3

In [11]:
setup_bank_database()
DB_PATH = 'data/bank_data.db'

✓ Database initialized
✓ Generated 500 account balance records
✓ Generated 300 loan status records

SAMPLE ACCOUNT BALANCES
ID: 3501, Account: ACC125639, Balance: $669,756.27, Updated: 2023-12-02 14:23:26
ID: 3502, Account: ACC812227, Balance: $32,522.40, Updated: 2023-11-12 14:23:26
ID: 3503, Account: ACC429137, Balance: $865,617.82, Updated: 2025-03-19 14:23:26
ID: 3504, Account: ACC917440, Balance: $2,002.76, Updated: 2025-05-06 14:23:26
ID: 3505, Account: ACC412050, Balance: $160.42, Updated: 2025-05-28 14:23:26

SAMPLE LOAN STATUS
ID: 2101, Account: ACC556369, Loan: $207,638.56, Rate: 4.76%, Status: active, Updated: 2024-11-02 14:23:26
ID: 2102, Account: ACC320370, Loan: $107,228.81, Rate: 5.51%, Status: active, Updated: 2023-07-15 14:23:26
ID: 2103, Account: ACC649817, Loan: $43,739.74, Rate: 5.36%, Status: active, Updated: 2024-10-03 14:23:26
ID: 2104, Account: ACC938486, Loan: $48,507.28, Rate: 12.44%, Status: active, Updated: 2025-04-12 14:23:26
ID: 2105, Account: ACC957861, L

In [36]:
@tool
def get_account_balance(account_id:str) -> Dict[str, Any]:
    """
    Get the account balance for a specific account ID
    
    Args:
        account_id (str): The account ID to lookup. Provide only the string part of the account ID, e.g., '1234567890'.
        
    Returns:
        dict: Dictionary containing account information or None if not found
              Format: {'account_id': str, 'balance': float, 'last_updated': str}
    """
    if not account_id or not isinstance(account_id, str):
        return {"error": "Invalid account_id provided"}
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    print(account_id)
    try:
        cursor.execute('''
            SELECT account_id, balance, last_updated 
            FROM account_balance 
            WHERE account_id = ?
            ORDER BY last_updated DESC
            LIMIT 1
        ''', (account_id,))
        
        result = cursor.fetchone()
        print("HERE")
        
        if result:
            return {
                'account_id': result[0],
                'balance': result[1],
                'last_updated': result[2]
            }
        else:
            return None
            
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None
    finally:
        conn.close()

In [24]:

@tool
def get_loan_status(account_id: str) -> List[Dict[str, Any]]:
   """
   Get all loan information for a specific account ID
   
   Args:
       account_id (str): The account ID to lookup
       
   Returns:
       list: List of dictionaries containing loan information, empty list if none found
             Format: [{'id': int, 'account_id': str, 'loan_amount': float, 
                      'interest_rate': float, 'status': str, 'last_updated': str}]
   """
   conn = sqlite3.connect(DB_PATH)
   cursor = conn.cursor()
   
   try:
       cursor.execute('''
           SELECT id, account_id, loan_amount, interest_rate, status, last_updated 
           FROM loan_status 
           WHERE account_id = ?
           ORDER BY last_updated DESC
           LIMIT 1
       ''', (account_id,))
       
       results = cursor.fetchall()
       
       loans = []
       for row in results:
           loans.append({
               'id': row[0],
               'account_id': row[1],
               'loan_amount': row[2],
               'interest_rate': row[3],
               'status': row[4],
               'last_updated': row[5]
           })
       
       return loans
       
   except sqlite3.Error as e:
       print(f"Database error: {e}")
       return []
   finally:
       conn.close()


In [25]:
@tool
def get_nearest_branch(zip_code) -> Dict[str, Any]:
   """
   Get the nearest bank branch based on zip code
   
   Args:
       zip_code (str): The zip code to find nearest branch for
       
   Returns:
       dict: Dictionary containing branch information
             Format: {'branch_id': str, 'name': str, 'address': str, 'city': str, 
                     'state': str, 'zip_code': str, 'phone': str, 'distance_miles': float,
                     'hours': str, 'services': list}
   """
   # Mock branch data - in reality this would come from a database or API
   mock_branches = {
       # Major city zip codes and their branches
       '10001': {'branch_id': 'NYC001', 'name': 'Manhattan Central Branch', 'address': '123 Broadway', 'city': 'New York', 'state': 'NY'},
       '90210': {'branch_id': 'LA001', 'name': 'Beverly Hills Branch', 'address': '456 Rodeo Drive', 'city': 'Beverly Hills', 'state': 'CA'},
       '60601': {'branch_id': 'CHI001', 'name': 'Downtown Chicago Branch', 'address': '789 Michigan Ave', 'city': 'Chicago', 'state': 'IL'},
       '77001': {'branch_id': 'HOU001', 'name': 'Houston Main Branch', 'address': '321 Main Street', 'city': 'Houston', 'state': 'TX'},
       '33101': {'branch_id': 'MIA001', 'name': 'Miami Beach Branch', 'address': '654 Ocean Drive', 'city': 'Miami', 'state': 'FL'},
       '98101': {'branch_id': 'SEA001', 'name': 'Seattle Downtown Branch', 'address': '987 Pine Street', 'city': 'Seattle', 'state': 'WA'},
       '02101': {'branch_id': 'BOS001', 'name': 'Boston Financial District', 'address': '147 State Street', 'city': 'Boston', 'state': 'MA'},
       '30301': {'branch_id': 'ATL001', 'name': 'Atlanta Midtown Branch', 'address': '258 Peachtree St', 'city': 'Atlanta', 'state': 'GA'},
   }
   
   # Check if we have an exact match
   if zip_code in mock_branches:
       branch_data = mock_branches[zip_code]
       distance = round(random.uniform(0.5, 2.0), 1)  # Very close for exact zip match
   else:
       # For unknown zip codes, return a random nearby branch
       branch_data = random.choice(list(mock_branches.values()))
       distance = round(random.uniform(2.5, 15.0), 1)  # Further distance for non-exact matches
   
   # Generate additional branch details
   services = random.sample([
       'ATM', 'Drive-through', 'Safe Deposit Boxes', 'Notary Services',
       'Business Banking', 'Mortgage Services', 'Investment Consulting',
       'Currency Exchange', '24/7 Banking', 'Mobile Banking Support'
   ], k=random.randint(4, 7))
   
   hours_options = [
       'Mon-Fri: 9AM-5PM, Sat: 9AM-2PM',
       'Mon-Fri: 8AM-6PM, Sat: 9AM-3PM',
       'Mon-Thu: 9AM-4PM, Fri: 9AM-6PM, Sat: 9AM-1PM',
       '24/7 ATM Access, Lobby: Mon-Fri 9AM-5PM'
   ]
   
   return {
       'branch_id': branch_data['branch_id'],
       'name': branch_data['name'],
       'address': branch_data['address'],
       'city': branch_data['city'],
       'state': branch_data['state'],
       'zip_code': zip_code,
       'phone': f"({random.randint(200, 999)}) {random.randint(200, 999)}-{random.randint(1000, 9999)}",
       'distance_miles': distance,
       'hours': random.choice(hours_options),
       'services': services
   }

## 5 Set up our Crew AI Agent
There are 4 main things that we need to set up for our Crew AI agent.
* The model: we will be using Amazon Bedrock
* The Agents
* The Tasks
* The Crew

In [26]:
# Set up Bedrock LLM with CrewAI's LLM class
llm = LLM(
    model="bedrock/anthropic.claude-3-5-haiku-20241022-v1:0",
    temperature=0.7,
)


In [41]:
bank_helper = Agent(
    role="Banking assistant",
    goal="Help the customer with their banking questions",
    backstory="You are a helpful banking assistant. You are known for being nice and professional with customers.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
   
)

task = Task(
        description="Understand and answer the query:: {guardrailed_query}. If the query requires information that is not available, politely inform the user and ask them.",
        expected_output="A comprehensive, well-structured response that addresses the original query.",
        agent=bank_helper,
         tools=[get_loan_status, get_nearest_branch, get_account_balance]
    )

crew = Crew(
        agents=[bank_helper],
    tasks=[task],

)



## 6. Run the Agent
Now that we have set up our agent, we can run it. We will start by creating a Crew AI Agent and then running it with a sample input. Guardrails will be only applied to the input that goes into the agent, and not to the output of the agent. 



In [ ]:
query = "What is my account balance? my id is ACC812227"  # Use a test query

# Apply guardrails
guardrailed_input, metadata = apply_guardrails(query)

print(f"Original query: {query}")
print(f"Guardrailed input: {guardrailed_input}")
print(f"Guardrail action: {metadata.get('action', 'NONE')}")



if metadata.get('action') == "GUARDRAIL_INTERVENED":
    print("⚠️ Guardrail intervention detected!")
    print("Request blocked by safety filters.")
else:
    try:
        response = crew.kickoff(inputs={"guardrailed_query": guardrailed_input})

        print(f"Agent Response: {response}")
    except Exception as e:
        print(f"Error executing crew: {str(e)}")


Applying guardrail to content: What is my account balance? my id is ACC812227...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/botocore/crt/auth.py:59: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow().replace(


Original query: What is my account balance? my id is ACC812227
Guardrailed input: What is my account balance? my id is ACC812227
Guardrail action: NONE
# Agent: Banking assistant
## Task: Understand and answer the query:: What is my account balance? my id is ACC812227. If the query requires information that is not available, politely inform the user and ask them.


# Agent: Banking assistant
## Thought: I'll help you check the account balance for the account ID ACC812227.
Thought: I need to use the get_account_balance tool to retrieve the account balance.
## Using tool: get_account_balance
## Tool Input: 
"{\"account_id\": \"ACC812227\"}"
## Tool Output: 
{'account_id': 'ACC812227', 'balance': 32522.4, 'last_updated': '2023-11-12 14:23:26'}


# Agent: Banking assistant
## Final Answer: 
Your account balance for Account ID ACC812227 is $32,522.40. 

Additional Details:
- Account ID: ACC812227
- Current Balance: $32,522.40
- Balance Last Updated: November 12, 2023 at 2:23:26 PM

Is there

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/botocore/crt/auth.py:59: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow().replace(


Agent Response: Your account balance for Account ID ACC812227 is $32,522.40. 

Additional Details:
- Account ID: ACC812227
- Current Balance: $32,522.40
- Balance Last Updated: November 12, 2023 at 2:23:26 PM

Is there anything else I can help you with today?


In [ ]:
query = "Give me advice on where should I invest"  # Use a test query

# Apply guardrails
guardrailed_input, metadata = apply_guardrails(query)

print(f"Original query: {query}")
print(f"Guardrailed input: {guardrailed_input}")
print(f"Guardrail action: {metadata.get('action', 'NONE')}")



if metadata.get('action') == "GUARDRAIL_INTERVENED":
    print("⚠️ Guardrail intervention detected!")
    print("Request blocked by safety filters.")
else:
    try:
        response = crew.kickoff(inputs={"guardrailed_query": guardrailed_input})

        print(f"Agent Response: {response}")
    except Exception as e:
        print(f"Error executing crew: {str(e)}")


We can also apply guardrails to the output of the agent by using the `apply_guardrails` function on the output of the agent. This will ensure that the output of the agent is also safe and responsible.

In [ ]:
query = "What is my account balance? my id is ACC812227"  # Use a test query

# Apply guardrails
guardrailed_input, metadata = apply_guardrails(query)

print(f"Original query: {query}")
print(f"Guardrailed input: {guardrailed_input}")
print(f"Guardrail action: {metadata.get('action', 'NONE')}")



if metadata.get('action') == "GUARDRAIL_INTERVENED":
    print("⚠️ Guardrail intervention detected!")
    print("Request blocked by safety filters.")
else:
    try:
        response = crew.kickoff(inputs={"guardrailed_query": guardrailed_input})
        print(type(response))
        guardrailed_output, output_metadata = apply_guardrails(response.raw)
        print(f"Original output: {query}")
        print(f"Guardrailed output: {guardrailed_input}")
        print(f"Guardrail action: {metadata.get('action', 'NONE')}")
        if output_metadata.get('action') == "GUARDRAIL_INTERVENED":
            print("⚠️ Guardrail intervention detected in response!")
            print("Response blocked by safety filters.")
        else:
            response = guardrailed_output
            print(f"Agent Response: {response}")
    except Exception as e:
        print(f"Error executing crew: {str(e)}")


Applying guardrail to content: What is my account balance? my id is ACC812227...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/botocore/crt/auth.py:59: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow().replace(


Original query: What is my account balance? my id is ACC812227
Guardrailed input: What is my account balance? my id is ACC812227
Guardrail action: NONE
# Agent: Banking assistant
## Task: Understand and answer the query:: What is my account balance? my id is ACC812227. If the query requires information that is not available, politely inform the user and ask them.


# Agent: Banking assistant
## Thought: I'll help you check the account balance for the account ID ACC812227.
Thought: I need to use the get_account_balance tool to retrieve the account balance.
## Using tool: get_account_balance
## Tool Input: 
"{\"account_id\": \"ACC812227\"}"
## Tool Output: 
{'account_id': 'ACC812227', 'balance': 32522.4, 'last_updated': '2023-11-12 14:23:26'}


# Agent: Banking assistant
## Final Answer: 
Your account balance for Account ID ACC812227 is $32,522.40. 

Additional Details:
- Account ID: ACC812227
- Current Balance: $32,522.40
- Balance Last Updated: November 12, 2023 at 2:23:26 PM

Is there

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/botocore/crt/auth.py:59: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow().replace(


Agent Response: Your account balance for Account ID ACC812227 is $32,522.40. 

Additional Details:
- Account ID: ACC812227
- Current Balance: $32,522.40
- Balance Last Updated: November 12, 2023 at 2:23:26 PM

Is there anything else I can help you with today?
